In [10]:
import torch;
import torch.nn as nn;
import torch.optim as optim;
import torch.nn.functional as F;
from torch.utils.data import DataLoader;
import torchvision.datasets as datasets;
import torchvision.transforms as transforms;

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

print(device, torch.__version__)

mps 2.4.1


In [11]:
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.gru(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

In [13]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        print(epoch, batch_idx, data.shape, targets.shape)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()


0 0 torch.Size([64, 28, 28]) torch.Size([64])
0 1 torch.Size([64, 28, 28]) torch.Size([64])
0 2 torch.Size([64, 28, 28]) torch.Size([64])
0 3 torch.Size([64, 28, 28]) torch.Size([64])
0 4 torch.Size([64, 28, 28]) torch.Size([64])
0 5 torch.Size([64, 28, 28]) torch.Size([64])
0 6 torch.Size([64, 28, 28]) torch.Size([64])
0 7 torch.Size([64, 28, 28]) torch.Size([64])
0 8 torch.Size([64, 28, 28]) torch.Size([64])
0 9 torch.Size([64, 28, 28]) torch.Size([64])
0 10 torch.Size([64, 28, 28]) torch.Size([64])
0 11 torch.Size([64, 28, 28]) torch.Size([64])
0 12 torch.Size([64, 28, 28]) torch.Size([64])
0 13 torch.Size([64, 28, 28]) torch.Size([64])
0 14 torch.Size([64, 28, 28]) torch.Size([64])
0 15 torch.Size([64, 28, 28]) torch.Size([64])
0 16 torch.Size([64, 28, 28]) torch.Size([64])
0 17 torch.Size([64, 28, 28]) torch.Size([64])
0 18 torch.Size([64, 28, 28]) torch.Size([64])
0 19 torch.Size([64, 28, 28]) torch.Size([64])
0 20 torch.Size([64, 28, 28]) torch.Size([64])
0 21 torch.Size([64, 28

In [15]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'{num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

59347/60000 with accuracy 98.91
9861/10000 with accuracy 98.61
